<a href="https://colab.research.google.com/github/SparshKhandelwal103/VisionForge/blob/main/zenspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers accelerate scipy safetensors
!pip install tensorflow==2.17.1
!pip install tensorflow-hub==0.16.1
!pip install gradio
!pip install pillow

  Using cached tensorflow-2.18.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 MB 555.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 77.3 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.18.1 which is incompatible.


In [12]:
import os
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import gradio as gr
from datetime import datetime
from huggingface_hub import login

# Force CPU-only execution for TensorFlow
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.config.set_visible_devices([], 'GPU')

print("Setting up ZenSpark- AI Art Generator...")

# Global variables for models
sd_pipe = None
nst_model = None
device = None

def setup_stable_diffusion():
    """Initialize Stable Diffusion pipeline"""
    global sd_pipe, device
    try:
        try:
            login()
        except:
            print("Skipping Hugging Face login...")

        model_id = "runwayml/stable-diffusion-v1-5"
        device = "cuda" if torch.cuda.is_available() else "cpu"

        print(f"Loading Stable Diffusion on {device}...")
        sd_pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            safety_checker=None,
            requires_safety_checker=False
        )
        sd_pipe = sd_pipe.to(device)
        print("✅ Stable Diffusion loaded successfully")
        return True
    except Exception as e:
        print(f"❌ Error loading Stable Diffusion: {e}")
        return False

Setting up ZenSpark- AI Art Generator...


In [13]:
def setup_style_transfer():
    """Initialize Neural Style Transfer model"""
    global nst_model
    try:
        print("Loading Style Transfer model...")
        with tf.device('/CPU:0'):
            model_url = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
            nst_model = hub.load(model_url)
        print("✅ Style Transfer model loaded successfully")
        return True
    except Exception as e:
        print(f"❌ Error loading Style Transfer model: {e}")
        print("Will use fallback method...")
        return False

def generate_image(prompt, num_steps=20, guidance=7.5):
    """Generate image from text prompt"""
    if sd_pipe is None:
        return None, "Stable Diffusion model not loaded"

    try:
        print(f"Generating: {prompt[:50]}...")
        image = sd_pipe(
            prompt,
            num_inference_steps=num_steps,
            guidance_scale=guidance,
            height=512,
            width=512
        ).images[0]

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        save_path = f"generated_{timestamp}.png"
        image.save(save_path)
        print(f"✅ Image generated and saved as {save_path}")

        return image, None
    except Exception as e:
        error_msg = f"Image generation failed: {str(e)}"
        print(f"❌ {error_msg}")
        return None, error_msg

def preprocess_image(image):
    """Preprocess image for style transfer"""
    try:
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image.astype('uint8'))
        elif not isinstance(image, Image.Image):
            return None

        if image.mode != 'RGB':
            image = image.convert('RGB')

        image = image.resize((256, 256), Image.Resampling.LANCZOS)
        image_array = np.array(image) / 255.0
        image_tensor = tf.convert_to_tensor(image_array, dtype=tf.float32)
        image_tensor = tf.expand_dims(image_tensor, axis=0)

        return image_tensor
    except Exception as e:
        print(f"❌ Image preprocessing failed: {e}")
        return None

def simple_style_transfer(content_tensor, style_tensor, strength=0.8):
    """Fallback style transfer using statistical matching"""
    try:
        style_mean = tf.reduce_mean(style_tensor, axis=[1, 2], keepdims=True)
        style_std = tf.math.reduce_std(style_tensor, axis=[1, 2], keepdims=True)
        content_mean = tf.reduce_mean(content_tensor, axis=[1, 2], keepdims=True)
        content_std = tf.math.reduce_std(content_tensor, axis=[1, 2], keepdims=True)

        normalized = (content_tensor - content_mean) / (content_std + 1e-8)
        stylized = normalized * style_std + style_mean
        result = strength * stylized + (1 - strength) * content_tensor
        return tf.clip_by_value(result, 0.0, 1.0)
    except Exception as e:
        print(f"❌ Simple style transfer failed: {e}")
        return content_tensor

def apply_style_transfer(content_image, style_image, style_strength=0.8):
    """Apply neural style transfer to images"""
    try:
        print("Applying style transfer...")

        content_tensor = preprocess_image(content_image)
        style_tensor = preprocess_image(style_image)

        if content_tensor is None or style_tensor is None:
            return None, "Failed to preprocess images"

        with tf.device('/CPU:0'):
            if nst_model is not None:
                try:
                    stylized = nst_model(tf.constant(content_tensor), tf.constant(style_tensor))[0]
                    blended = style_strength * stylized + (1 - style_strength) * content_tensor
                    result_tensor = tf.clip_by_value(blended, 0.0, 1.0)
                    print("✅ Neural style transfer applied")
                except Exception as model_error:
                    print(f"Neural model failed: {model_error}")
                    print("Using fallback method...")
                    result_tensor = simple_style_transfer(content_tensor, style_tensor, style_strength)
            else:
                result_tensor = simple_style_transfer(content_tensor, style_tensor, style_strength)
                print("✅ Fallback style transfer applied")

        result_array = (result_tensor[0].numpy() * 255).astype(np.uint8)
        result_image = Image.fromarray(result_array)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        save_path = f"stylized_{timestamp}.png"
        result_image.save(save_path)
        print(f"✅ Stylized image saved as {save_path}")

        return result_image, None

    except Exception as e:
        error_msg = f"Style transfer failed: {str(e)}"
        print(f"❌ {error_msg}")
        return None, error_msg

def process_zenspark_request(prompt, style_image, style_strength, num_steps, guidance_scale):
    """Main processing function for Power ArtForge"""

    if not prompt or not prompt.strip():
        return None, None, "❌ Please enter a text prompt"

    if style_image is None:
        return None, None, "❌ Please upload a style image"

    if not (0.0 <= style_strength <= 1.0):
        style_strength = 0.8

    base_image, gen_error = generate_image(prompt, num_steps, guidance_scale)
    if base_image is None:
        return None, None, f"❌ Generation failed: {gen_error}"

    styled_image, style_error = apply_style_transfer(base_image, style_image, style_strength)
    if styled_image is None:
        return base_image, None, f"⚠️ Style transfer failed: {style_error}"

    return base_image, styled_image, "✅ ZenSpark processing complete!"

# Initialize models
print("🎨 Initializing ZenSpark models...")
sd_success = setup_stable_diffusion()
nst_success = setup_style_transfer()

if not sd_success:
    print("⚠️ Warning: Stable Diffusion failed to load")
if not nst_success:
    print("⚠️ Warning: Style Transfer will use fallback method")

# Create Custom Gradio Interface with Equal-Sized Components
with gr.Blocks(
    title="🎨 ZenSpark - AI Art Generator & Style Transfer",
    theme=gr.themes.Soft(),
    css="""
    .gradio-container {
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    #prompt_center {
        text-align: center;
        max-width: 800px;
        margin: 0 auto;
    }
    #style_image_large {
        min-height: 400px !important;
    }
    #style_image_large .gr-image {
        min-height: 400px !important;
    }
    # ... rest of your existing CSS
    """

) as demo:

    # Centered heading
    with gr.Column(elem_id="heading_center"):
        gr.Markdown("""
        # 🎨 ZenSpark - AI Art Generator & Style Transfer
        **Transform your ideas into stunning artwork!**

        Enter a text prompt, upload a style image, and watch the magic happen!
        """)

    # Prompt text centered and alone at the top
    with gr.Column(elem_id="prompt_center"):
        prompt_input = gr.Textbox(
            label="✨ Text Prompt",
            placeholder="Describe the image you want to create (e.g., 'A magical forest with glowing mushrooms')",
            lines=3,
            max_lines=5
        )

    # Style reference image and style controls - EQUAL SIZE
    with gr.Row():
        # Left column - Style reference image
        with gr.Column(scale=1):
            style_image_input = gr.Image(
                label="🎨 Style Reference Image",
                type="pil",
                image_mode="RGB",
                sources=["upload", "webcam"],  # Enable both upload and webcam
                height=400,  # Make it larger
                width=400,   # Make it larger
                elem_id="style_image_large"  # Apply custom CSS
        )

        # Right column - Style intensity + Advanced settings (EQUAL SIZE)
        with gr.Column(scale=1):
            style_intensity_input = gr.Slider(
                minimum=0.0,
                maximum=1.0,
                value=0.8,
                step=0.1,
                label="🎯 Style Intensity",
                info="Higher values = stronger style effect"
            )

            # Advanced settings moved here and integrated
            with gr.Group():
                gr.Markdown("### 🔧 Advanced Settings")
                num_steps_input = gr.Slider(
                    minimum=10,
                    maximum=50,
                    value=20,
                    step=5,
                    label="⚡ Generation Steps",
                    info="More steps = higher quality (but slower)"
                )
                guidance_scale_input = gr.Slider(
                    minimum=1.0,
                    maximum=15.0,
                    value=7.5,
                    step=0.5,
                    label="🎛️ Guidance Scale",
                    info="Higher values = closer to prompt"
                )

    # Submit button
    submit_btn = gr.Button("🎨 Generate", variant="primary", size="lg")

    # Generated image and stylized result side by side (equal size)
    with gr.Row():
        generated_output = gr.Image(label="🖼️ Generated Image", type="pil")
        stylized_output = gr.Image(label="🎨 Stylized Result", type="pil")

    # Status at the bottom with centered text
    with gr.Column(elem_id="status_center"):
        status_output = gr.Textbox(label="📋 Status", interactive=False)

    # Event handler
    submit_btn.click(
        fn=process_zenspark_request,
        inputs=[
            prompt_input,
            style_image_input,
            style_intensity_input,
            num_steps_input,
            guidance_scale_input
        ],
        outputs=[generated_output, stylized_output, status_output]
    )

    # Example section
    gr.Markdown("""
    ## 🌟 Example Prompts to Try:
    - "A serene mountain landscape at golden hour with a crystal clear lake"
    - "A futuristic cyberpunk city with neon lights and flying cars"
    - "A majestic dragon soaring through storm clouds"
    - "A cozy library filled with ancient books and warm candlelight"

    *Tip: Try famous art styles like Van Gogh, Picasso, or Japanese woodblock prints as style references!*
    """)

🎨 Initializing ZenSpark models...


Loading Stable Diffusion on cuda...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

✅ Stable Diffusion loaded successfully
Loading Style Transfer model...
✅ Style Transfer model loaded successfully


In [ ]:
# Launch the application
if __name__ == "__main__":
    print("🚀 Launching ARTBIX with your custom layout...")
    try:
        demo.launch(
            share=True,
            debug=True,
            server_name="0.0.0.0",
            server_port=7860,
            show_error=True
        )
    except Exception as launch_error:
        print(f"❌ Failed to launch Gradio: {launch_error}")
        print("💡 Try running on a different port:")
        demo.launch(share=True, server_port=7861)

🚀 Launching ARTBIX with your custom layout...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ddc71b2bb4d61f64ab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Generating: ronaldo...


  0%|          | 0/45 [00:00<?, ?it/s]

✅ Image generated and saved as generated_20250724_155536.png
Applying style transfer...
✅ Neural style transfer applied
✅ Stylized image saved as stylized_20250724_155539.png
Generating: batman...


  0%|          | 0/25 [00:00<?, ?it/s]

✅ Image generated and saved as generated_20250724_160443.png
Applying style transfer...
✅ Neural style transfer applied
✅ Stylized image saved as stylized_20250724_160444.png
Generating: f1 car...


  0%|          | 0/45 [00:00<?, ?it/s]

✅ Image generated and saved as generated_20250724_160626.png
Applying style transfer...
✅ Neural style transfer applied
✅ Stylized image saved as stylized_20250724_160627.png
